# Use External Databases

##### In this notebook we make use of external sources such as US IRS data and the Canadian Census Bureau to turn zip codes into meaningful data, in particular the **household median income**. 

https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi

https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi

For Canada: 

https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/inc-rev/Table.cfm?Lang=Eng&T=102&PR=0&D1=1&RPP=25&SR=1&S=108&O=D


Understanding Canadian postal codes:

https://www.postalcodesincanada.com/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils

%matplotlib inline

# United States of America

In [2]:
irs = utils.chunk_loader('census_data/16zpallagi.csv', index_col=False)

In [3]:
irs.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,1,815440,477700,105350,221200,440830,1296920,...,367320,330066,0,0,0,0,63420,51444,711580,1831661
1,1,AL,0,2,495830,211930,142340,128890,272440,996240,...,398050,984297,0,0,0,0,74090,110889,416090,1173463
2,1,AL,0,3,263390,83420,137870,36340,154880,584000,...,253180,1349246,0,0,0,0,64000,143060,195130,543284
3,1,AL,0,4,167190,29420,124060,10610,99700,421720,...,165830,1425430,0,0,0,0,45020,128920,117410,381329
4,1,AL,0,5,217440,20240,188080,4880,129410,601040,...,216720,3922449,390,155,60,19,82940,423629,126130,506526


In [4]:
irs.shape

(179796, 147)

In [5]:
#checking my address for fun
irs[irs['zipcode']==94104]

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
15277,6,CA,94104,1,250,200,40,30,160,290,...,130,220,0,0,0,0,60,90,130,278
15278,6,CA,94104,2,190,150,40,0,100,250,...,170,602,0,0,0,0,50,112,120,239
15279,6,CA,94104,3,140,110,0,30,80,180,...,140,1086,0,0,0,0,50,168,80,229
15280,6,CA,94104,4,110,80,20,0,70,150,...,110,1369,0,0,0,0,40,286,70,319
15281,6,CA,94104,5,230,160,60,0,140,350,...,230,5652,0,0,0,0,70,873,140,791
15282,6,CA,94104,6,370,170,170,20,320,760,...,370,292539,240,3051,270,20469,120,5150,140,27134


In [6]:
irs.columns

Index(['STATEFIPS', 'STATE', 'zipcode', 'agi_stub', 'N1', 'mars1', 'MARS2',
       'MARS4', 'PREP', 'N2',
       ...
       'N10300', 'A10300', 'N85530', 'A85530', 'N85300', 'A85300', 'N11901',
       'A11901', 'N11902', 'A11902'],
      dtype='object', length=147)

From the dataframe documentation we care about two features:
- zipcode
- AGI stub: Size of adjusted gross income
    - 1 = 1 under 25,000
    - 2 = 25,000 under 50,000
    - 3 = 50,000 under 75,000
    - 4 = 75,000 under 100,000
    - 5 = 100,000 under 200,000
    - 6 = 200,000 or more
Note that all incomes reported here are in United States dollar.

We can use this to build a sliding scale and assign a median household income for each zip code. The rationale is that businesses definetly need to know what the income is in their area of operation in order to study their business model, products, pricing, etc...

In [7]:
irs_median = irs.groupby(by=['zipcode'], as_index=True).median()
irs_median = irs_median['agi_stub']
irs_median.head()

zipcode
0       3.5
1001    3.5
1002    3.5
1003    3.0
1005    3.5
Name: agi_stub, dtype: float64

In [8]:
irs_median.unique()

array([3.5, 3. , 4. , 2.5])

In [9]:
#conver to dict
usa_median = irs_median.to_dict()

#get mean:
usa_median['mean'] = irs_median.mean()

# Canada

For Canada some differences with USA:
- Data reported in Canadian dollars
- Use 0.76 Canadian dollars for each USA dollar
- Postal code format differs
- Data might be less granular
- Encoding might differ
- Consistent with IRS take income before taxes

The Canadian data is a lot less granular than USA however even at the provincial level we know that income will matter a lot.

In [10]:
#get encoding of file via commandline
! chardetect census_data/canada/98-402-X2016006-T1-CSD-Eng.csv

census_data/canada/98-402-X2016006-T1-CSD-Eng.csv: Windows-1252 with confidence 0.73


In [11]:
#download data
canada = pd.read_csv('census_data/canada/98-402-X2016006-T1-CSD-Eng.csv',
                     encoding='windows-1252')

#drop columns that have missing values
canada = canada.dropna(axis=1, how='any')

#rename relevant columns
canada = canada.rename(columns={'Geographic name, Province or territory': 'province', 
                                'Median household total income (2015 constant dollars), 2015': 'median_2015',
                                'Number of households, 2016': 'household_count_2016', 
                                'Number of households, 2005': 'household_count_2005'})


canada.head()

,Geographic code,Geographic name,Geographic type,province,"Geographic code, Province or territory","Geographic code, Census division",Global non-response rate,Data quality flag,Household type,household_count_2016,median_2015,"Median household after-tax income (2015 constant dollars), 2015"
0,1001113,Trepassey,Town,Newfoundland and Labrador,10,1001,4.5,0,Total – Household type including census family...,245,53888,48192
1,1001113,Trepassey,Town,Newfoundland and Labrador,10,1001,4.5,0,Census-family households,165,71424,63424
2,1001113,Trepassey,Town,Newfoundland and Labrador,10,1001,4.5,0,Households consisting of only one census f...,160,72192,63744
3,1001113,Trepassey,Town,Newfoundland and Labrador,10,1001,4.5,0,"One couple, with or without children in ...",140,77568,68608
4,1001113,Trepassey,Town,Newfoundland and Labrador,10,1001,4.5,0,"One couple, without children in their ...",105,73472,64128


In [12]:
#check cities
canada['Geographic name'].unique()

array(['Trepassey', 'Division No.  1, Subd. U', 'Cape Broyle', ...,
       'Cambridge Bay', 'Gjoa Haven', 'Taloyoak'], dtype=object)

In [13]:
#check provinces
canada['province'].unique()

array(['Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia',
       'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan',
       'Alberta', 'British Columbia', 'Yukon', 'Northwest Territories',
       'Nunavut'], dtype=object)

In [14]:
canada.shape

(33372, 12)

In [15]:
canada.columns

Index(['Geographic code', 'Geographic name', 'Geographic type', 'province',
       'Geographic code, Province or territory',
       'Geographic code, Census division', 'Global non-response rate',
       'Data quality flag', 'Household type', 'household_count_2016',
       'median_2015',
       'Median household after-tax income (2015 constant dollars), 2015'],
      dtype='object')

In [16]:
canada_sigma = canada.groupby(by=['province'], as_index=True).sum()
canada_pop = canada_sigma['household_count_2016'].to_dict()
canada_pop

{'Alberta': 5904215,
 'British Columbia': 7027060,
 'Manitoba': 1869200,
 'New Brunswick': 1231240,
 'Newfoundland and Labrador': 837910,
 'Northwest Territories': 54990,
 'Nova Scotia': 1523205,
 'Nunavut': 36505,
 'Ontario': 19911580,
 'Prince Edward Island': 227715,
 'Quebec': 13148470,
 'Saskatchewan': 1586060,
 'Yukon': 53140}

In [17]:
def weighted_income(df, 
                    prov, 
                    pop_dict=canada_pop, 
                    prov_col='province',
                    median_col='median_2015', 
                    pop_col='household_count_2016', 
                    conversion_rate=0.76):
    #get dataframe of province
    df_prov = df[df[prov_col]==prov].copy()
    
    #number of households in province
    population = pop_dict[prov]
    
    #get fraction contribution of each region
    df_prov['weights'] = df_prov[pop_col]/population
    
    #get weighted median
    weighted_median = (df_prov[median_col] * df_prov['weights']).sum()
    
    #convert to american dollars
    weighted_median *= conversion_rate
    
    return int(weighted_median)

In [18]:
def income_to_cat(income):
    if income <= 25000:
        return 1
    elif income <=50_000:
        return 2
    elif income <=75_000:
        return 3
    elif income <=100_000:
        return 4
    elif income <=200_000:
        return 5
    elif income >200_000:
        return 6

In [19]:
#test it out
print(weighted_income(canada, 'Yukon'))

76384


In [20]:
canada_median = {}
canada_median_list = []

for province in canada_pop:
    median = income_to_cat(weighted_income(canada, province))
    canada_median[province] = median
    canada_median_list.append(median)
    
canada_median['mean'] = np.mean(canada_median_list)

canada_median

{'Alberta': 4,
 'British Columbia': 3,
 'Manitoba': 3,
 'New Brunswick': 3,
 'Newfoundland and Labrador': 3,
 'Northwest Territories': 5,
 'Nova Scotia': 3,
 'Nunavut': 4,
 'Ontario': 3,
 'Prince Edward Island': 3,
 'Quebec': 3,
 'Saskatchewan': 3,
 'Yukon': 4,
 'mean': 3.3846153846153846}

# Postal Codes to Median Incomes

In [21]:
def canada_post_to_prov(postal):
    first = postal[0]
    
    if first in 'A':
        return 'Newfoundland and Labrador'
    elif first in 'B':
        return 'Nova Scotia'
    elif first in 'C':
        return 'Prince Edward Island'
    elif first in 'E':
        return 'New Brunswick'
    elif first in 'GHJ':
        return 'Quebec'
    #note W is part of ontario
    elif first in 'KLMNPW':
        return 'Ontario'
    elif first in 'R':
        return 'Manitoba'
    elif first in 'S':
        return 'Saskatchewan'
    elif first in 'T':
        return 'Alberta'
    elif first in 'V':
        return 'British Columbia'
    elif first in 'X':
        return 'Northwest Territories'
    elif first in 'Y':
        return 'Yukon'
    else:
        return None

In [22]:
#location of file
business_dir = 'data/business.json'

#download businsess
df_bus = utils.chunk_loader(business_dir, read_limit=-1)
df_bus.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",None,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


In [23]:
def postal_to_cat(postal, canada_median=canada_median, usa_median=usa_median):
    #check for missing values and if so return mean category for usa
    if len(postal) == 0:
        return usa_median['mean']
    
    elif not postal.isdigit():
        try:
            #make upper case in case of typo error
            province = canada_post_to_prov(postal[0].upper())
            return canada_median[province]
        #Some postal codes are UK
        except KeyError:
            return 0
    elif postal.isdigit():
        #note the type conversion
        try:
            median = usa_median[int(postal)]
            return median
        except KeyError:
            return usa_median['mean']
    else:
        return 0

In [24]:
#get reduced dataframe
df_bus_postal = df_bus[['business_id', 'postal_code']].copy()

#convert postal codes to categorical variable
df_bus_postal['median_income'] = df_bus_postal['postal_code'].apply(postal_to_cat)

#drop postal code after usage
df_bus_postal = df_bus_postal.drop(columns=['postal_code'])

df_bus_postal.head()

,business_id,median_income
0,1SWheh84yJXfytovILXOAQ,3.5
1,QXAEGFB4oINsVuTFxEYKFQ,3.0
2,gnKjwL_1w79qoiV3IC_xQQ,3.5
3,xvX2CttrVhyG2z1dFg_0xw,3.5
4,HhyxOkGAM07SRYtlQ4wMFQ,3.5


In [25]:
#save the work
df_bus_postal.to_csv('data/cleaned/business_median_income.csv')